# DSL development

## Imports

In [1]:
import random
import numpy as np
from arc25.dsl import *
from arc25.training_tasks import RandomDrawingTask

## Drawing samples

I want to generate training data so the model can learn how to draw things. It is very easy to generate
synthetic training data and learning about it should provide the model an understanding of the 2d structure of the grids.

In [2]:
img = create_img(np.random.randint(1, 10, 2), color=random.randint(0, 9))
draw_line(img, (0, 0), (1, 2), color=random.randint(0, 9))
img

34444
43344
44444
44444
44444
44444
44444

In [3]:
task = RandomDrawingTask()

In [4]:
print(task.sample().code)

draw_pixel(img, point=(1, 5), color=8)
draw_horizontal_line(img, y=4, color=6)

